In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import random

%matplotlib inline

In [3]:
import sys

sys.path.append("./")
sys.path.append("../code/")
sys.path.append("./code/")

import time

In [4]:
from typing import List, Tuple, Union

In [5]:
from loguru import logger

# Data Loader

In [6]:
from bps_numerical.preprocessing import merge_gene_phenotype, standardize_gene_data

In [7]:
CSV_GENE = "/Users/nishparadox/dev/uah/nasa-impact/gene-experiments/data/OneDrive_1_3-21-2022/gen.csv"

CSV_PHENOTYPE = "/Users/nishparadox/dev/uah/nasa-impact/gene-experiments/data/OneDrive_1_3-21-2022/meta.csv"

In [8]:
# df_merged = merge_gene_phenotype(standardize_gene_data(CSV_GENE), CSV_PHENOTYPE)

In [9]:
df_genes = standardize_gene_data(CSV_GENE)

2022-07-20 12:37:24.161 | INFO     | bps_numerical.preprocessing:standardize_gene_data:27 - Standardizing gene data into proper format.


In [10]:
df_genes.head()

,Sample,ENSMUSG00000099250,ENSMUSG00000064339,ENSMUSG00000099021,ENSMUSG00000064351,ENSMUSG00000065037,ENSMUSG00000064337,ENSMUSG00000100862,ENSMUSG00000029368,ENSMUSG00000064370,...,ENSMUSG00000013367,ENSMUSG00000008153,ENSMUSG00000013083,ENSMUSG00000013155,ENSMUSG00000012819,ENSMUSG00000010307,ENSMUSG00000011837,ENSMUSG00000012889,ENSMUSG00000001642,ENSMUSG00000006019
1,GLDS_242_Mmus_C57_6J_LVR_GC_C2_Rep4_G5,4.565105,4.486067,4.22359,4.137298,3.773993,3.86423,3.093246,4.149449,3.38502,...,-0.748711,0.514245,-0.834259,-0.637977,-1.04317,0.360869,0.374517,-0.665341,0.748225,0.347033
2,GLDS_48_Mmus_C57_6J_LVR_FLT_C_Rep2_M26,0.563454,3.140386,0.778641,4.384875,0.745577,3.01786,4.037345,4.180711,4.089054,...,-0.15882,0.151228,-1.032761,-0.42141,-1.151805,0.205813,0.746601,-0.813002,0.670417,0.730899
3,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_ERCC_Rep5_M30,3.968279,3.490494,3.984012,3.721062,3.526439,2.489441,2.644076,4.093958,2.904823,...,-0.599768,-0.157375,-1.130212,-0.677432,-1.094534,0.10989,0.389537,-0.571371,0.592057,0.711531
4,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_noERCC_Rep2_M26,3.980689,3.333291,3.988481,3.494274,3.49009,2.370999,2.394222,4.108144,2.68952,...,-0.739483,-0.103737,-1.071226,-0.535766,-1.029838,0.178408,0.361334,-0.453689,0.657613,0.623739
5,GLDS_245_Mmus_C57_6T_LVR_GC_LAR_Rep4_G5,4.991286,4.222969,4.533385,3.712087,4.184306,3.324107,2.698993,4.135948,2.888685,...,-0.592125,0.2066,-1.019552,-0.606019,-1.187632,0.356565,0.547004,-0.900965,0.880465,0.414951


In [11]:
samples = df_genes.pop("Sample")

In [12]:
df_genes = df_genes.astype(float)

In [13]:
df_genes.iloc[0].dtype

dtype('float64')

# Feature Selection

In [14]:
from bps_numerical.clustering import CorrelationClusterer
from bps_numerical.feature_selection import FirstFeatureSelector, KRandomizedFeatureSelector

In [15]:
clusterer = CorrelationClusterer(
    list(df_genes.columns),
    cutoff_threshold=0.3,
    debug=False
)

In [16]:
fs = FirstFeatureSelector(clusterer=clusterer)
fs = KRandomizedFeatureSelector(clusterer=clusterer, k_features=2)

In [17]:
cols_genes = fs.select_features(df_genes)

2022-07-20 12:48:04.509 | DEBUG    | bps_numerical.clustering:cluster:62 - Computing correlation for (6832, 25000)
2022-07-20 12:48:21.400 | INFO     | bps_numerical.clustering:_cluster:111 - Clustering in progress...
2022-07-20 12:48:41.035 | DEBUG    | bps_numerical.clustering:cluster:82 - Took 19.635257959365845 seconds to form 9268 clusters...


In [18]:
# cols_genes = list(df_genes.columns)

In [19]:
len(cols_genes)

11216

In [20]:
cols_genes[:10]

['ENSMUSG00000055320',
 'ENSMUSG00000115200',
 'ENSMUSG00000049717',
 'ENSMUSG00000025007',
 'ENSMUSG00000016458',
 'ENSMUSG00000020340',
 'ENSMUSG00000044177',
 'ENSMUSG00000111380',
 'ENSMUSG00000087400',
 'ENSMUSG00000087543']

In [21]:
# len(df_genes.columns)

# Data Prep

In [22]:
df_merged = merge_gene_phenotype(
    pd.concat([samples, df_genes[cols_genes]], axis=1),
    CSV_PHENOTYPE,
    "Sample",
)

2022-07-20 12:49:11.103 | INFO     | bps_numerical.preprocessing:merge_gene_phenotype:52 - Merging gene-phenotype dataframes...


In [23]:
df_merged.shape

(6832, 11229)

In [24]:
df_merged.head()

,Sample,age,animalreturn,dataset,condition,duration,gender,libPrep,mission,preservation,...,ENSMUSG00000107841,ENSMUSG00000067722,ENSMUSG00000027859,ENSMUSG00000089990,ENSMUSG00000084067,ENSMUSG00000094551,ENSMUSG00000065770,ENSMUSG00000037259,ENSMUSG00000029372,ENSMUSG00000104927
0,GLDS_242_Mmus_C57_6J_LVR_GC_C2_Rep4_G5,10,LAR,GLDS_242,GC,33,male,ribodepleted,RR9,immediate,...,-0.848108,1.024172,0.365140,-1.123242,-1.214940,-1.272680,-1.362091,-0.775916,0.017438,-1.180251
1,GLDS_48_Mmus_C57_6J_LVR_FLT_C_Rep2_M26,16,ISST,GLDS_48,FLT,37,female,polyA,RR1_NASA,carcass,...,-0.864303,1.180727,0.386947,-1.130186,-1.011914,-0.641275,-1.319513,-0.139219,-0.026540,-0.846743
2,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_ERCC_Rep5_M30,16,ISST,GLDS_168,FLT,37,female,ribodepleted,RR1_NASA,carcass,...,-0.942326,0.928198,0.125523,-1.180625,-1.196364,-0.819959,-1.256005,-0.166665,-0.271712,-1.008163
3,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_noERCC_Rep2_M26,16,ISST,GLDS_168,FLT,37,female,ribodepleted,RR1_NASA,carcass,...,-0.719939,0.936706,-0.024223,-1.120430,-1.059349,-1.076152,-1.302438,-0.267983,-0.243608,-0.997188
4,GLDS_245_Mmus_C57_6T_LVR_GC_LAR_Rep4_G5,32,LAR,GLDS_245,GC,29,female,ribodepleted,RR6,immediate,...,-0.917089,0.844935,0.531730,-0.961090,-1.200634,-1.096963,-1.215244,-0.178179,-0.029066,-0.969156


In [25]:
len(cols_genes)

11216

# Bulk Trainer

Here we train in bulk and then unify feature sets. These features are then used in the downstream classification.

In [27]:
from bps_numerical.classification import (
    SinglePhenotypeClassifier,
    MultiPhenotypeIsolatedClassifier
)
from bps_numerical.classification.classifiers import BulkTrainer
from bps_numerical.classification.feature_scorers import PhenotypeFeatureScorer, UnifiedFeatureScorer

In [29]:
clf_condition = SinglePhenotypeClassifier(cols_genes, "condition")
clf_strain = SinglePhenotypeClassifier(cols_genes, "strain")
clf_gender = SinglePhenotypeClassifier(cols_genes, "gender")
clf_mission = SinglePhenotypeClassifier(cols_genes, "mission")
clf_animal_return = SinglePhenotypeClassifier(cols_genes, "animalreturn")

multi_trainer = MultiPhenotypeIsolatedClassifier(
    cols_genes=cols_genes,
    classifiers=[
        clf_condition,
        clf_strain,
    ],
    debug=True,
)

bulk_trainer = BulkTrainer(
    cols_genes=cols_genes,
    classifiers=[multi_trainer],
    n_runs=5,
)

In [30]:
results_bulk = bulk_trainer.train(df_merged)

  0%|                                                                                                                                                                  | 0/2 [00:00<?, ?it/s]2022-07-20 12:49:36.802 | INFO     | bps_numerical.classification.classifiers:train:321 - Training for phenotype=condition
2022-07-20 12:49:36.805 | DEBUG    | bps_numerical.classification.classifiers:train:330 - Target phenotype stats:: FLT    2757
GC     2708
Name: condition, dtype: int64
2022-07-20 12:49:36.806 | DEBUG    | bps_numerical.classification.classifiers:train:333 - n genes = 11216 || Labels -> ['condition_FLT', 'condition_GC']
2022-07-20 12:50:22.655 | DEBUG    | bps_numerical.classification.classifiers:fit:72 - Training took 45.8493447303772 seconds.

 50%|█████████████████████████████████████████████████████████████████████████████                                                                             | 1/2 [00:45<00:45, 45.97s/it]2022-07-20 12:50:22.773 | INFO     | bps_numerica

  0%|                                                                                                                                                                  | 0/2 [00:00<?, ?it/s]2022-07-20 12:55:21.138 | INFO     | bps_numerical.classification.classifiers:train:321 - Training for phenotype=condition
2022-07-20 12:55:21.141 | DEBUG    | bps_numerical.classification.classifiers:train:330 - Target phenotype stats:: FLT    2777
GC     2688
Name: condition, dtype: int64
2022-07-20 12:55:21.141 | DEBUG    | bps_numerical.classification.classifiers:train:333 - n genes = 11216 || Labels -> ['condition_FLT', 'condition_GC']
2022-07-20 12:56:09.410 | DEBUG    | bps_numerical.classification.classifiers:fit:72 - Training took 48.26797914505005 seconds.

 50%|█████████████████████████████████████████████████████████████████████████████                                                                             | 1/2 [00:48<00:48, 48.38s/it]2022-07-20 12:56:09.522 | INFO     | bps_numeric

In [32]:
len(results_bulk)

5

In [34]:
results_bulk

[[{'condition': {'labels': ['condition_FLT', 'condition_GC'],
    'train_score': 1.0,
    'test_score': 0.9992684711046086,
    'confusion_matrix': array([[720,   0],
           [  1, 646]]),
    'classification_report': {'condition_FLT': {'precision': 0.9986130374479889,
      'recall': 1.0,
      'f1-score': 0.9993060374739764,
      'support': 720},
     'condition_GC': {'precision': 1.0,
      'recall': 0.9984544049459042,
      'f1-score': 0.9992266047950503,
      'support': 647},
     'micro avg': {'precision': 0.9992684711046086,
      'recall': 0.9992684711046086,
      'f1-score': 0.9992684711046086,
      'support': 1367},
     'macro avg': {'precision': 0.9993065187239945,
      'recall': 0.999227202472952,
      'f1-score': 0.9992663211345134,
      'support': 1367},
     'weighted avg': {'precision': 0.9992694857077923,
      'recall': 0.9992684711046086,
      'f1-score': 0.9992684420509588,
      'support': 1367},
     'samples avg': {'precision': 0.9992684711046086,
  

In [35]:
type(bulk_trainer.classifiers[0][0]), bulk_trainer.classifiers[0][0]

(bps_numerical.classification.classifiers.MultiPhenotypeIsolatedClassifier,
 <bps_numerical.classification.classifiers.MultiPhenotypeIsolatedClassifier at 0x300e02400>)

In [42]:
PhenotypeFeatureScorer(
    bulk_trainer.classifiers[1][0]
).get_features(
    top_k=500,
    ignore_zeros=True,
    normalize=True
)

[('ENSMUSG00000115767', 0.012971930438652635),
 ('ENSMUSG00000085148', 0.005460385931655765),
 ('ENSMUSG00000020340', 0.00026181925568380393)]

In [66]:
feature_scores = UnifiedFeatureScorer(bulk_trainer, debug=False).get_features(top_k=500)
feature_scores

[('ENSMUSG00000020340', 0.0008434034116362454),
 ('ENSMUSG00000028540', 0.007964181946590543),
 ('ENSMUSG00000028864', 0.015690930653363466),
 ('ENSMUSG00000049717', 0.008011775091290474),
 ('ENSMUSG00000055320', 0.0017011471863952465),
 ('ENSMUSG00000085148', 0.005460385931655765),
 ('ENSMUSG00000095280', 0.0091994924005121),
 ('ENSMUSG00000095771', 0.0037970044650137424),
 ('ENSMUSG00000098013', 0.02343968825880438),
 ('ENSMUSG00000105361', 0.009157964228506899),
 ('ENSMUSG00000114479', 0.0052322885021567345),
 ('ENSMUSG00000115200', 0.007245425600558519),
 ('ENSMUSG00000115767', 0.024832588736899197)]

In [67]:
# PhenotypeFeatureScorer(bulk_trainer, debug=True).get_features(
#     top_k=500,
#     ignore_zeros=True,
#     normalize=True
# )

In [68]:
cols_genes_subset, _ = zip(*feature_scores)

In [75]:
cols_genes_subset

('ENSMUSG00000020340',
 'ENSMUSG00000028540',
 'ENSMUSG00000028864',
 'ENSMUSG00000049717',
 'ENSMUSG00000055320',
 'ENSMUSG00000085148',
 'ENSMUSG00000095280',
 'ENSMUSG00000095771',
 'ENSMUSG00000098013',
 'ENSMUSG00000105361',
 'ENSMUSG00000114479',
 'ENSMUSG00000115200',
 'ENSMUSG00000115767')

# Ranker

To further narrow down the gene space, we could train N different isolated classifiers for a specific
phenotype and then use those genes for training only that phenotype later.

In [ ]:
from bps_numerical.classification.feature_scorers import GeneRanker

In [ ]:
ranker = GeneRanker(
    cols_genes, 
    phenotype="condition",
    n_runs = 5,
    debug=True,
)

In [ ]:
features = ranker.get_features(df_merged, test_size=0.2, top_k=500, ignore_zeros=True, normalize=True)

In [ ]:
len(features)

In [ ]:
fts, _ = zip(*features)
fts

# Trainer

In [ ]:
# single phenotype

# model = xgboost.XGBClassifier()
clf = SinglePhenotypeClassifier(
    cols_genes=cols_genes,
    phenotype="condition",
#     model = model
)

In [ ]:
# tracker_single = clf.train(df_merged)

In [ ]:
# tracker_single

# Multiple phenotypes

In [ ]:
clf_condition = SinglePhenotypeClassifier(cols_genes, "condition")
clf_strain = SinglePhenotypeClassifier(cols_genes, "strain")
# clf_gender = SinglePhenotypeClassifier(cols_genes, "gender")
# clf_mission = SinglePhenotypeClassifier(cols_genes, "mission")
# clf_animal_return = SinglePhenotypeClassifier(cols_genes, "animalreturn")
trainer = MultiPhenotypeIsolatedClassifier(
    cols_genes=cols_genes_subset,
    classifiers=[
        clf_condition,
        clf_strain,
    ],
    debug=True,
)

In [ ]:
tracker_multi = trainer.train(df_merged)

In [ ]:
tracker_multi

In [ ]:
tracker_multi["condition"]

# Feature Scorer

In [ ]:
from bps_numerical.classification.feature_scorers import PhenotypeFeatureScorer

In [ ]:
for clf in trainer.classifiers:
    print(clf.phenotype, len(PhenotypeFeatureScorer(clf).get_features(top_k=500, ignore_zeros=True)))

In [ ]:
PhenotypeFeatureScorer(clf).get_features(top_k=500, ignore_zeros=True, normalize=True)

In [ ]:
len(PhenotypeFeatureScorer(trainer).get_features(top_k=500, ignore_zeros=False))

In [ ]:
# if we ignore 0-score features
len(PhenotypeFeatureScorer(*trainer.classifiers).get_features(top_k=500, ignore_zeros=True))

In [ ]:
list(map(lambda f: f[0], PhenotypeFeatureScorer(clf_condition, clf_strain).get_features(top_k=500, ignore_zeros=True, normalize=True)))

### permutation

In [ ]:
import itertools

In [ ]:
def compute_permuted_scores(*classifiers, ignore_zeros: bool = True, top_k: int = 500):
    def _powerset(items):
        for sl in itertools.product(*[[[], [i]] for i in items]):
            yield {j for i in sl for j in i}
    
    res = {}
    for objs in _powerset(classifiers):
        if len(objs) < 2:
            continue
        labels = tuple(map(lambda clf: clf.phenotype, objs))
        res[labels] = PhenotypeFeatureScorer(*objs).get_features(top_k=top_k, ignore_zeros=ignore_zeros, normalize=True)
    return res

In [ ]:
permuted_ =  compute_permuted_scores(*trainer.classifiers, top_k=1000, ignore_zeros=True)

In [ ]:
dict(map(lambda p: (p[0], (len(p[1]), p[1])), permuted_.items()))

In [ ]:
dict(map(lambda p: (p[0], len(p[1])), permuted_.items()))

# Plot Top features

In [ ]:
def plot_features(features: List[Tuple[str, float]], view_slicer:int = 75):
    df_top_k = pd.DataFrame(features[:view_slicer], columns=["gene", "importance"])
    fig = px.bar(
        df_top_k,
        x="importance",
        y="gene",
        title=f"{view_slicer} features",
        orientation="h",
        height=1600,
        width=1000,
    #     text_auto=True,
    )
    # fig.update_traces(width=3)
    fig.update_layout(yaxis = dict(tickfont = dict(size=7)))
    fig.show()

In [ ]:
plot_features(
    PhenotypeFeatureScorer(clf_mission, clf_strain).get_features(top_k=500, ignore_zeros=True, normalize=True)
)

# Ranking

In [ ]:
from typing import List, Tuple
from tqdm import tqdm
from pprint import pprint

In [ ]:
from bps_numerical.classification import SinglePhenotypeClassifier, MultiPhenotypeIsolatedClassifier

In [ ]:
from bps_numerical.classification.feature_scorers import PhenotypeFeatureScorer

In [ ]:
class GeneRanker:
    def __init__(
        self,
        cols_genes: List[str],
        phenotype: str,
        n_runs: int = 3,
        debug: bool = False
    ) -> None:

        self.classifiers=[
            SinglePhenotypeClassifier(cols_genes=cols_genes, phenotype=phenotype, debug=debug)
            for _ in range(n_runs)
        ]
        self.results = []
        self.debug = debug
        self.phenotype = phenotype
    
    def get_features(self, data: pd.DataFrame, test_size: float = 0.2, **kwargs) -> dict:
        self.results = [clf.train(data, test_size) for clf in self.classifiers]
        
        ignore_zeros = kwargs.get("ignore_zeros", True)
        normalize = kwargs.get("normalize", True)
        top_k = kwargs.get("top_k", 500)

        if self.debug:
            pprint(self.results)
            self._debug_plot_hist(**kwargs)
                
        
        return PhenotypeFeatureScorer(*self.classifiers).get_features(
            top_k=top_k,
            ignore_zeros=ignore_zeros,
            normalize=normalize
        )
    
    def _debug_plot_hist(self, **kwargs):
        if not self.debug:
            return
        ignore_zeros = kwargs.get("ignore_zeros", True)
        normalize = kwargs.get("normalize", True)
        top_k = kwargs.get("top_k", 500)
        nfeatures = []
        for clf in self.classifiers:
            features = PhenotypeFeatureScorer(clf).get_features(
                top_k=top_k,
                ignore_zeros=ignore_zeros,
                normalize=normalize
            )
            nfeatures.append(len(features))
            logger.debug(f"{clf.phenotype} | {len(features)}")

        _df_counter = pd.DataFrame(enumerate(nfeatures), columns=["run", "n_feature"])
        fig = px.bar(
            _df_counter,
            x="run",
            y="n_feature",
            title=f"run vs n_feature for {self.phenotype}",
            text_auto=True,
        )
        fig.update_layout(yaxis = dict(tickfont = dict(size=kwargs.get("debug_font_size", 7))))
        fig.show()

In [ ]:
ranker = GeneRanker(
    cols_genes, 
    phenotype="condition",
    n_runs = 5,
    debug=True,
)

In [ ]:
results = ranker.rank_genes(df_merged)

In [ ]:
# for clf in ranker.classifiers:
#     print(clf.phenotype, len(PhenotypeFeatureScorer(clf).get_features(top_k=500, ignore_zeros=True)))

In [ ]:
# PhenotypeFeatureScorer(*ranker.classifiers).get_features(top_k=500, ignore_zeros=True, normalize=True)